<h1> Problem 3 - Largest Prime Factor </h1>

The prime factors of $13195$ are $5$, $7$, $13$, and $29$. 

What is the largest prime factor of the number $600851475143$?

<h2> Factorisation </h2>

The first part of the solution will use Fermat's Factorisation Method. For details, refer to [wikipedia Fermat's Factorisation Method](https://en.wikipedia.org/wiki/Fermat's_factorization_method). 

Based on the representation of an odd integer as a difference of two squares,
$$
    N = a^2 - b^2 = (a+b) (a-b)
$$
If neither $a$ nor $b$ is equal to $1$, then it is a proper factorisation of N

In [2]:
fn gcd(mut a: usize, mut b: usize) -> usize{
    let mut remainder;
    while b!= 0 {
        remainder = a % b;
        a = b;
        b = remainder;
    };
    a
}

In [3]:
// I don't know how to declare 10^-9 in const for Rust 
const EPSILON: f64 = 0.000001;

/// Function to test if input is square
fn is_square(n: f64) -> bool{
    //  Get square root of term
    let n_sqrt = n.sqrt();
    //  Get whole number portion of squareroot
    let whole = n_sqrt as usize;
    let whole = whole as f64;
    
    if n_sqrt < whole + EPSILON && n_sqrt > whole - EPSILON{
        return true;
    };
    false
}

/// Naive Fermat factorisation method 
fn naive(n: usize) -> (usize, usize) {
    let n_f = n as f64;
    let mut a = n_f.sqrt().ceil();
    let mut b2 = a*a - n_f;
    
    while ! is_square(b2){
        a += 1.0;
        b2 = a*a - n_f
    };
    
//     println!("{}", b2);
    
    let out: (usize, usize) = ((a-b2.sqrt()) as usize, (a + b2.sqrt()) as usize);
    out
}

<!-- This method is great! But it doesn't find the prime factors, just the factorisation of the number -->

<h2> Find Prime Numbers </h2>

The second part of this solution uses the Seive of Eratosthenes [wikipedia Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes). It is an ancient method to find all of the prime numbers up to $n$.

It has a time complexity of $\mathcal{O}(n \log{\log{n}})$.

Annoyingly, I can't find a function that is equivalent to `np.argwhere(x==True)` to get the indices of array with `true`. Therefore, I've created a function to do this for me. 

N.B.: This Seive method is such that only the final output array can be used to determine which numbers are prime. Therefore, it can't be determined within the loop (of the seive) which numbers will be prime

In [4]:
// Interestingly, the rust lang book says that i32 should be default but here the RustFmt, keeps recommending
// that I use usize instead so I'll use that :) 

fn seive(n: usize) -> Vec<bool> {
    // Initialise aray of boolean values from 2 to n (shifted in implementation)
    // i.e. index 0 = 2, index 1 = 3, ... etc
    let mut out = vec![true; (n-2) as usize];
    
    // Max iteration number = sqrt(n)
    let n_sqrt: usize = ((n as f64).sqrt() + 1.0) as usize;
//     println!("{}", n_sqrt);
    
    // For i = 2, ..., sqrt(n)
    for i in 2..n_sqrt {
        // if that index is true
        if out[i-2] == true {
            // Iterate over j = i^2, i^2 + i, ..., n
            let mut k: usize = 0;
            let mut j: usize = i*i + k*i; 
//             println!("{}", n);
            while j < n {
                out[j-2] = false;
//                 println!("{}, {}", k, j);
                k += 1;
                j = i*i + k*i;
            }
        }
    }
    out
}


use std::iter

fn get_prime_numbers(input: Vec<bool>) -> Vec<usize> {
    let mut prime_numbers: Vec<usize> = Vec::new();
    let num: usize = input.len();
    
    // Iterate over the entire array (output of seive)
    for i in 0..num {
        // If it is true => prime number
        if input[i] == true {
            // Add the prime number to the output vector
            // The indices are offset by 2
             prime_numbers.push(i+2);
        }
    }
    prime_numbers
}

fn get_largest_prime_factor(number: usize, primes: Vec<usize>) -> usize {
    // Get the total length of the vector of primes 
    // This is the total number of primes for the largest multiple
    let n: usize = primes.len();
    
    // Initialise output
    let mut out: usize = 0;
    
    // Iterate starting from the back to get largest multiple
    for i in (0..n).rev() {
        // If the number we desire can be divided nicely by the prime number, 
        // Deem it to be the largest prime factor and return (exit function)
        if number % primes[i] == 0 {
            return primes[i];
        }
    }
    out
}

<h2> How does this method work? </h2>

1. Take the number and split it into its two largest multiples, $(a-b)(a+b)$. This uses the Fermat Factorisation Method 
2. Using the larger of the two multiples, $a+b$, find all of the prime numbers up to $a+b$. As I am unable to find an in-built method within Rust to get the indices of the vector that are true, `get_prime_numbers` is a function that does that. 
3. The largest prime factor is found by, starting with the largest prime number, divide the desired number by the prime. The first number that is able to divide the desired number without any remainder is the largest prime factor.

**Why bother finding the two largest multiples?**

I don't know if it works out to be computationally cheaper. But, intuitively, it makes sense to first make the desired number smaller, then find all the primes up to that number. 

**Why the largest of the two multiples?**

That's to be safe. It's possible that the largest prime factor will lie in within $[a-b, a+b]$.

In [5]:
// let desired_number: usize = 13195;
let desired_number: usize = 600851475143;

// By definition, naive returns a-b, a+b
// Therefore, x2 is always larger
let (x1, x2) = naive(desired_number);

// Using the larger of the two multiples, find all the primes of it
let seive_output: Vec<bool> = seive(x2);
// println!("{:?}", seive_output);
let prime_nums: Vec<usize> = get_prime_numbers(seive_output);
// println!("{:?}", prime_nums);

// Get the largest prime factor
let num: usize = get_largest_prime_factor(desired_number, prime_nums);
println!("The largest prime factor is {}", num);

The largest prime factor is 6857
